<b>Данный блокнот производит обработку изображений. Мы хотим получить эмбеддинги изображений с помощью трансформера 'facebook/dinov2-base'

In [70]:
from transformers import AutoImageProcessor, AutoModel
import numpy as np
from PIL import Image
import requests
import torch
from tqdm import tqdm
import os
import csv

In [ ]:
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')
input_path = 'Ozon/' # Путь до картинок относительно блокнота
output_path = 'test_image_embeddings.csv' # Путь до выходного csv-шника

In [72]:
from logging import exception
def get_dinov2_embeddings(image):
  try:
    # Автоматическая предобработка до 224x224
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Берем [CLS] token как embedding всего изображения
    embeddings = outputs.last_hidden_state[:, 0]  # [1, 768]
    return embeddings.numpy(), True

  except Exception as e:
    print(f"Ошибка обработки {image_path}: {e}")
    return None, False

In [ ]:
def process_folder(folder_path, output_csv):
    # Поиск всех изображений
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
    image_paths = []
    for file in os.listdir(folder_path):
        file_ext = os.path.splitext(file.lower())[1]
        if file_ext in image_extensions:
            full_path = os.path.join(folder_path, file)
            image_paths.append(full_path)

    print(f"Найдено {len(image_paths)} изображений для обработки")

    if not image_paths:
        print("Не найдено изображений для обработки!")
        return

    # Собираем данные в список для создания DataFrame
    data_rows = []
    successful = 0
    
    for image_path in tqdm(image_paths, desc="Обработка изображений"):
        embedding, success = get_dinov2_embeddings(Image.open(image_path).convert('RGB'))
        if success:
            filename = os.path.basename(image_path)
            row = {
                'id': filename.split('.')[0],
                'filepath': image_path
            }
            # Добавляем каждый элемент эмбеддинга как отдельную колонку
            for i, emb_val in enumerate(embedding):
                row[f'emb_{i}'] = emb_val
            
            data_rows.append(row)
            successful += 1

    # Создаем DataFrame и сохраняем в CSV
    df = pd.DataFrame(data_rows)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"Успешно обработано: {successful}/{len(image_paths)} изображений")
    print(f"Результаты сохранены в: {output_csv}")
    print(f"DataFrame создан с формой: {df.shape}")
    
    return df

In [80]:
process_folder(input_path, output_path)

Найдено 5 изображений для обработки


Обработка изображений: 100%|██████████| 5/5 [00:04<00:00,  1.04it/s]

Успешно обработано: 5/5 изображений
Результаты сохранены в: output.csv
